In [2]:
# install this before running the notebook
#!pip install h2o
#!pip install requests
#!pip install tabulate
#!pip install future

# Required for plotting:
#!pip install matplotlib
#!pip uninstall h2o
#!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
# !pip install autogluon.tabular[all]


In [244]:
import pandas as pd
import tensorflow as tf
import numpy as np
from pathlib import Path
from autogluon.tabular import TabularDataset, TabularPredictor

def generate_predictions(input_data=None,model=4):
    
    input_data = input_data.numpy()
    
    
    save_location = Path(r'C:\\Users\\mfavre4\\Desktop\\NN_python')
    #print(save_location)
    i = int(model)
    predictor_path = save_location / f'agModels-R_{i}'
    #print(predictor_path)
    predictor = TabularPredictor.load(predictor_path, require_version_match=False, require_py_version_match=False)
   
    model_to_use = predictor.model_names()[1]
    if 'Torch' in model_to_use:
        #print('Loading torch NN ....', end=' ')
        # print(model_to_use)
        torch_model = predictor._trainer.load_model(model_to_use)
        model_info = torch_model.get_info()
#        print(torch_model.lead)
        #print('Success')

    #print('\n\nInput data: ')
    #print(input_data)
    #print(type(input_data))
    #print(input_data[0])
    # if input_data == None:
    #     print('No input data found')
    #     data = pd.read_csv('X_test_25.csv')
    #     unnamed_cols = [col for col in data.columns if 'Unnamed' in col]
    #     if len(unnamed_cols) > 0:
    #         data.drop(unnamed_cols, axis=1, inplace=True)
    # else:
    #print('Loading input data....')
    #data_dict = {'rotor_posn':input_data[0],
    #             'stator_exc_0':input_data[1],
    #             'stator_exc_U':input_data[2],
    #             'stator_exc_D':input_data[3],
    #              }

    #print('data_dict: ', data_dict)
    
    #data = pd.DataFrame(data_dict, index=[0])
    data = pd.DataFrame(input_data, columns=['rotor_posn', 'stator_exc_0', 'stator_exc_U', 'stator_exc_D'])
        
    prediction = torch_model.predict(data)

    prediction_df = pd.Series(prediction)
    prediction_ts = tf.convert_to_tensor(prediction)
    predictions = tf.cast(prediction_ts, dtype=tf.float32)

    #print('Sample predictions:')
    #print(prediction_df.head())
    
    prediction_df.to_csv('prediction_25.csv')

    return predictions#prediction.tolist()



In [17]:
import pandas as pd
from pathlib import Path
import shutil

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [18]:
current_dir = Path('.')
data = pd.read_csv(current_dir.absolute() / 'concatDataset.csv')
data.head()

,I_Coil_S_1,I_Coil_S_2,I_Coil_S_3,I_Coil_S_4,I_Coil_S_5,I_Coil_S_6,I_Coil_S_7,I_Coil_S_8,I_Coil_S_9,I_Coil_S_10,I_Coil_S_11,I_Coil_S_12,I_Coil_S_13,I_Coil_S_14,I_Coil_S_15,I_Coil_S_16,I_Coil_S_17,I_Coil_S_18,I_Coil_S_19,I_Coil_S_20,I_Coil_S_21,I_Coil_S_22,I_Coil_S_23,I_Coil_S_24,I_Coil_S_25,I_Coil_S_26,I_Coil_S_27,I_Coil_S_28,I_Coil_S_29,I_Coil_S_30,T_R_Core,T_R_Core_2,T_R_Core_3,T_R_Core_4,T_R_Core_5,T_R_Core_6,T_R_Core_7,T_R_Core_8,T_R_Core_9,T_R_Core_10,T_R_Core_11,T_R_Core_12,T_R_Core_13,T_R_Core_14,T_R_Core_15,T_R_Core_16,T_R_Core_17,T_R_Core_18,T_R_Core_19,T_R_Core_20,T_R_Core_21,T_R_Core_22,T_R_Core_23,T_R_Core_24,T_R_Core_25,T_R_Core_26,T_R_Core_27,T_R_Core_28,Rotor_posn,Net_Torque
0,838.911750,759.710510,151.419436,1615.056972,743.575343,391.038655,1261.606568,242.427231,1517.168814,1235.880686,1136.968086,1644.552590,1299.043478,373.753114,1154.363677,968.558844,475.046566,1728.383799,695.624587,1451.133024,805.648549,1142.269616,200.665408,1902.587556,601.935652,40.713244,1532.422884,1856.700587,418.592063,955.759736,15.479226,-42.847403,6.064716,-12.895680,18.252286,-29.389699,11.222207,-26.003880,-21.294832,31.174374,-29.326844,112.638421,-24.577963,24.606552,4.854151,-57.440308,27.289150,-117.839598,37.453612,-47.446869,17.398821,-26.851500,-28.688864,-13.777332,-11.467442,122.742396,-14.451273,10.036495,-1.200000,-65.087081
1,63.159728,1145.927954,1657.822801,304.390164,494.859837,1807.131972,1049.181833,94.704052,489.101852,426.085510,358.871208,1578.618173,314.545535,612.220245,636.828373,622.404727,404.759696,1805.421913,1690.866852,1461.239642,280.030105,1460.276274,5.973251,268.499457,870.218774,570.007014,1128.917385,703.853609,597.595272,1904.930449,11.229471,-12.536256,30.446560,-103.580160,20.265228,17.454621,19.543563,-38.937267,-10.905801,13.979060,-8.572148,112.500900,-8.759866,25.504916,0.290484,-40.982595,41.326342,-100.186679,26.732960,-66.229009,9.483419,-44.946565,-8.622409,27.807482,-15.351617,82.259023,-10.616755,31.961057,-1.175758,0.557958
2,604.895390,1687.818115,1012.687243,1502.671404,405.011494,1454.313326,589.616445,1840.587726,21.227596,353.216915,1263.009765,1345.094191,1366.001321,110.808211,729.066266,1374.909432,153.261000,279.744488,658.661727,1519.840001,1756.416608,1286.093196,501.047096,1929.647805,974.581579,129.650555,1607.304410,820.807789,1824.520892,473.856855,17.637504,-31.529522,15.031434,-49.240416,15.424546,5.433440,13.185292,38.785119,-19.140526,20.559278,-7.332849,97.164002,-16.685405,5.809139,6.244711,-75.382638,-5.592654,-13.735950,50.292172,-13.282855,17.478178,-22.011852,-25.213906,8.814699,-19.264357,130.770247,-13.454053,36.582125,-1.151515,167.344904
3,17.840346,1759.438076,1574.016330,212.658270,330.123381,1857.333290,970.880979,224.690144,1995.096232,1716.554497,938.142312,231.825997,574.401833,312.836168,1135.919141,1026.209542,1396.933967,1093.977710,1367.545965,1917.831175,823.392415,595.425860,1878.078629,1205.817297,597.432528,858.841403,343.724064,882.827033,445.642094,1678.188694,12.994437,-6.987048,24.382430,-80.895173,16.929088,35.475627,7.852790,-11.329015,-17.989747,53.259204,-40.250328,25.070360,-0.386462,7.051670,10.609738,-65.145659,22.312188,-42.883271,46.302679,-69.812219,4.260714,57.851876,-41.122330,13.443905,-17.304365,29.278734,-2.543550,5.425467,-1.127273,-24.148261
4,309.189519,1211.884724,1241.877155,1765.598706,958.032189,180.943129,1683.015986,1804.044797,1496.436842,1253.418474,1899.860847,1739.772124,860.744173,9.266228,829.254146,643.718123,404.240796,1258.558023,1777.273983,630.647459,1237.191367,459.165193,803.252614,937.639814,1289.839030,338.787254,61.456967,1785.606285,1092.452883,835.650610,15.827835,-28.320735,36.028130,-32.938019,21.500328,-36.722931,-10.237192,39.495440,-44.268432,59.057567,-10.266856,76.100659,-17.603028,-6.133252,5.726628,-45.872483,28.380931,-30.701357,20.523078,-3.909566,0.900474,17.781660,-19.684223,48.454667,-18.190603,31.816934,8.748907,-21.019876,-1.103030,84.474686


In [19]:
data.shape
data.rename({'T_R_Core' : 'T_R_Core_1'}, axis='columns', inplace=True)
data[['T_R_Core_1', 'I_Coil_S_1', 'Rotor_posn']].describe()

,T_R_Core_1,I_Coil_S_1,Rotor_posn
count,22400.000000,22400.000000,2.240000e+04
mean,55.209673,996.029242,3.092764e-17
std,48.230066,578.535337,6.997991e-01
min,-110.847693,0.013550,-1.200000e+00
25%,25.347180,494.145012,-6.000000e-01
50%,50.223036,992.259122,1.006140e-16
75%,82.158135,1496.497232,6.000000e-01
max,243.900608,1999.980027,1.200000e+00


Data prep

In [20]:
stator_pole_position = {f'S_{sp+1}' : round(36/30 * sp, 1) for sp in range(30)}
# stator_pole_position
rotor_pole_position = {f'R_{rp+1}' : round(36/28 * rp, 2) for rp in range(28)}
rotor_pole_position

{'R_1': 0.0,
 'R_2': 1.29,
 'R_3': 2.57,
 'R_4': 3.86,
 'R_5': 5.14,
 'R_6': 6.43,
 'R_7': 7.71,
 'R_8': 9.0,
 'R_9': 10.29,
 'R_10': 11.57,
 'R_11': 12.86,
 'R_12': 14.14,
 'R_13': 15.43,
 'R_14': 16.71,
 'R_15': 18.0,
 'R_16': 19.29,
 'R_17': 20.57,
 'R_18': 21.86,
 'R_19': 23.14,
 'R_20': 24.43,
 'R_21': 25.71,
 'R_22': 27.0,
 'R_23': 28.29,
 'R_24': 29.57,
 'R_25': 30.86,
 'R_26': 32.14,
 'R_27': 33.43,
 'R_28': 34.71}

In [21]:
def find_closest_key(stator_dict, target_value):
    closest_key = min(stator_dict, key=lambda x: abs(stator_dict[x] - target_value))
    return closest_key

In [22]:
rotor_stator_pairs = {}
for rotor in rotor_pole_position:
    rotor_id = int(rotor[2:])
    target_value = rotor_pole_position[rotor]
    target_stator = find_closest_key(stator_pole_position, target_value)
    print(f'For rotor: {rotor_id} at posn: {target_value}, \
          closest stator: {target_stator} at : {stator_pole_position[target_stator]}')
    rotor_stator_pairs[rotor] = target_stator

For rotor: 1 at posn: 0.0,           closest stator: S_1 at : 0.0
For rotor: 2 at posn: 1.29,           closest stator: S_2 at : 1.2
For rotor: 3 at posn: 2.57,           closest stator: S_3 at : 2.4
For rotor: 4 at posn: 3.86,           closest stator: S_4 at : 3.6
For rotor: 5 at posn: 5.14,           closest stator: S_5 at : 4.8
For rotor: 6 at posn: 6.43,           closest stator: S_6 at : 6.0
For rotor: 7 at posn: 7.71,           closest stator: S_7 at : 7.2
For rotor: 8 at posn: 9.0,           closest stator: S_8 at : 8.4
For rotor: 9 at posn: 10.29,           closest stator: S_10 at : 10.8
For rotor: 10 at posn: 11.57,           closest stator: S_11 at : 12.0
For rotor: 11 at posn: 12.86,           closest stator: S_12 at : 13.2
For rotor: 12 at posn: 14.14,           closest stator: S_13 at : 14.4
For rotor: 13 at posn: 15.43,           closest stator: S_14 at : 15.6
For rotor: 14 at posn: 16.71,           closest stator: S_15 at : 16.8
For rotor: 15 at posn: 18.0,           cl

In [23]:
rotor_stator_pairs

{'R_1': 'S_1',
 'R_2': 'S_2',
 'R_3': 'S_3',
 'R_4': 'S_4',
 'R_5': 'S_5',
 'R_6': 'S_6',
 'R_7': 'S_7',
 'R_8': 'S_8',
 'R_9': 'S_10',
 'R_10': 'S_11',
 'R_11': 'S_12',
 'R_12': 'S_13',
 'R_13': 'S_14',
 'R_14': 'S_15',
 'R_15': 'S_16',
 'R_16': 'S_17',
 'R_17': 'S_18',
 'R_18': 'S_19',
 'R_19': 'S_20',
 'R_20': 'S_21',
 'R_21': 'S_22',
 'R_22': 'S_23',
 'R_23': 'S_25',
 'R_24': 'S_26',
 'R_25': 'S_27',
 'R_26': 'S_28',
 'R_27': 'S_29',
 'R_28': 'S_30'}

In [24]:
data_ML_all = {}
for rotor in rotor_pole_position:
    rotor_id = int(rotor[2:])
    data_ML = pd.DataFrame(columns = ['rotor_posn', 'stator_exc_0', 'stator_exc_U', 'stator_exc_D', 'torque_rotor'])
    if rotor_id > 2 and rotor_id < 28- 2:
        print(f'For rotor: {rotor}', end='...')
        stator_id = int(rotor_stator_pairs[rotor][2:])
        data_ML['rotor_posn'] = stator_pole_position[f'S_{stator_id}'] - (data['Rotor_posn'] + rotor_pole_position[rotor])
        data_ML['stator_exc_0'] = data['I_Coil_' + rotor_stator_pairs[rotor]]
        data_ML['stator_exc_U'] = data[f'I_Coil_S_{stator_id + 1}']
        data_ML['stator_exc_D'] = data[f'I_Coil_S_{stator_id - 1}']
        data_ML['torque_rotor'] = data[f'T_R_Core_{rotor_id}']
        print(f"Min: {data_ML['rotor_posn'].min():.2f}, Max: {data_ML['rotor_posn'].max():.2f}, Range: {data_ML['rotor_posn'].max() - data_ML['rotor_posn'].min(): .2f}, \
        Torque: {data_ML['torque_rotor'].max() - data_ML['torque_rotor'].min():.2f}, Shape: {data_ML.shape[0]}")
        data_ML_all[rotor] = data_ML
        # break

For rotor: R_3...Min: -1.37, Max: 1.03, Range:  2.40,         Torque: 125.46, Shape: 22400
For rotor: R_4...Min: -1.46, Max: 0.94, Range:  2.40,         Torque: 316.17, Shape: 22400
For rotor: R_5...Min: -1.54, Max: 0.86, Range:  2.40,         Torque: 121.28, Shape: 22400
For rotor: R_6...Min: -1.63, Max: 0.77, Range:  2.40,         Torque: 309.51, Shape: 22400
For rotor: R_7...Min: -1.71, Max: 0.69, Range:  2.40,         Torque: 122.71, Shape: 22400
For rotor: R_8...Min: -1.80, Max: 0.60, Range:  2.40,         Torque: 296.58, Shape: 22400
For rotor: R_9...Min: -0.69, Max: 1.71, Range:  2.40,         Torque: 126.50, Shape: 22400
For rotor: R_10...Min: -0.77, Max: 1.63, Range:  2.40,         Torque: 310.07, Shape: 22400
For rotor: R_11...Min: -0.86, Max: 1.54, Range:  2.40,         Torque: 119.68, Shape: 22400
For rotor: R_12...Min: -0.94, Max: 1.46, Range:  2.40,         Torque: 312.41, Shape: 22400
For rotor: R_13...Min: -1.03, Max: 1.37, Range:  2.40,         Torque: 128.19, Shape: 2

In [25]:
data_ML_all['R_3'].describe()

,rotor_posn,stator_exc_0,stator_exc_U,stator_exc_D,torque_rotor
count,22400.000000,22400.000000,22400.000000,22400.000000,22400.000000
mean,-0.170000,996.900911,1003.020502,1002.405288,-1.167778
std,0.699799,577.604178,575.561780,576.696840,20.780160
min,-1.370000,0.103686,0.111777,0.097202,-62.869201
25%,-0.770000,494.269513,500.168919,508.436744,-15.648767
50%,-0.170000,1003.104630,996.666789,999.583852,-1.775934
75%,0.430000,1490.050507,1504.467747,1501.528661,13.153638
max,1.030000,1999.957406,1999.987138,1999.988031,62.593093


Make a predition

In [101]:
in_test = data_ML_all['R_3'].drop('torque_rotor', axis=1)

pred = generate_predictions(in_test,25)
print(type(pred))


############################## WARNING ##############################
	Predictor Version: 1.0.0
	Current Version:   1.1.0
############################## WARNING ##############################

Found 2 mismatches between original and current metadata:


C:\Users\mfavre4\Desktop\NN_python
C:\Users\mfavre4\Desktop\NN_python\agModels-R_25
<class 'list'>


In [27]:
target = data_ML_all['R_3']['torque_rotor'].to_frame()

In [28]:
from sklearn.metrics import r2_score
r2_score(target, pred)*100

89.71500111523105

In [245]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from keras.callbacks import Callback

# Assuming you have your data stored in a pandas DataFrame
X = data_ML_all['R_3'].drop(columns=[ 'stator_exc_0', 'stator_exc_U', 'stator_exc_D'])
y = data_ML_all['R_3'][[ 'stator_exc_0', 'stator_exc_U', 'stator_exc_D']]

# Split the data into training and testing sets
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

# Normalize the data
'''
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = pd.DataFrame(scaler_X.fit_transform(X_train), columns=X_train.columns)
X_test =  pd.DataFrame(scaler_X.transform(X_test), columns=X_train.columns)

y_train = pd.DataFrame(scaler_y.fit_transform(y_train), columns=y_train.columns)
y_test = pd.DataFrame(scaler_y.transform(y_test), columns=y_test.columns)
'''

'\nscaler_X = StandardScaler()\nscaler_y = StandardScaler()\n\nX_train = pd.DataFrame(scaler_X.fit_transform(X_train), columns=X_train.columns)\nX_test =  pd.DataFrame(scaler_X.transform(X_test), columns=X_train.columns)\n\ny_train = pd.DataFrame(scaler_y.fit_transform(y_train), columns=y_train.columns)\ny_test = pd.DataFrame(scaler_y.transform(y_test), columns=y_test.columns)\n'

![Getting Started](./Loss_custom.png)

In [249]:
def custom_loss(batch, y_true, y_pred):
    print(y_pred.shape)
    #Loss called for each batch 
    #Figuring which batch we are in to have the right samples
    #Should I be passing the rotor position ? y_pred.shape -> (32,4)
    test_size = 0.2
    batch_iteration  = batch.batch_number
    batch_size = y_true.shape[0]
    
    start_idx = batch_iteration*batch_size
    end_idx = start_idx + batch_size  
    
    
    #Choosing samples from the validation or training set according to the phase
    if y_pred.shape[0] == int(y.shape[0] * test_size):
        #Validation phase
        #rotor_pos = X_test['rotor_posn'][start_idx:end_idx].values
        rotor_pos = tf.convert_to_tensor(X_test['rotor_posn'][start_idx:end_idx].values, dtype=tf.float32)
        true_torque = tf.convert_to_tensor(X_test['torque_rotor'][start_idx:end_idx].values, dtype=tf.float32)
    else:
        #Training phase
        #rotor_pos = X_train['rotor_posn'][start_idx:end_idx].values
        rotor_pos = tf.convert_to_tensor(X_train['rotor_posn'][start_idx:end_idx].values, dtype=tf.float32)
        true_torque =tf.convert_to_tensor(X_train['torque_rotor'][start_idx:end_idx].values, dtype=tf.float32)
    print('slice size custom', rotor_pos.shape)
    
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    # Expand dimensions of rotor_pos and concatenate with y_pred
    rotor_pos_expanded = tf.expand_dims(rotor_pos, axis=1)
    
    y_pred_with_rotor = tf.concat([rotor_pos_expanded, y_pred], axis=1)
    
    
    #tf.compat.v1.disable_eager_execution()
    #with tf.compat.v1.Session() as sess:
    #    y_pred_with_rotor_np = sess.run(y_pred_with_rotor)    
        
    #y_pred_with_rotor_df = pd.DataFrame(y_pred_with_rotor_np, columns=['rotor_posn', 'col2', 'col3', 'col4'])
    
    # Ensure y_pred_with_rotor has 4 columns
    assert y_pred_with_rotor.shape[1] == 4, "y_pred_with_rotor should have 4 columns"

    
    # Use tf.py_function to call the generate_predictions function
    #inverse_nn_predictions = tf.py_function(func=lambda y_pred_with_rotor_df: tf.convert_to_tensor(generate_predictions(y_pred_with_rotor_df, 25), dtype=tf.float32), inp=[y_pred_with_rotor], Tout=tf.float32)
    
    
    model=25
    inverse_nn_predictions = tf.py_function(func=lambda y_pred_with_rotor: generate_predictions(y_pred_with_rotor,model), inp=[y_pred_with_rotor], Tout=tf.float32)

    tf.print(inverse_nn_predictions)
    #inverse_nn_predictions = tf.py_function(func=generate_predictions, inp=[y_pred_with_rotor, 25], Tout=tf.float32)
    # Calculate the loss
    loss = tf.reduce_mean(tf.abs(true_torque - inverse_nn_predictions))
    
    batch.batch_number = batch.batch_number +1
    return loss


In [250]:


# Define input shape
input_shape = (X.shape[1])  # Assuming input dimension is 100, change accordingly

# Input layer
inputs = Input(shape=input_shape)

# Hidden layers
hidden1 = Dense(64, activation='tanh')(inputs)
hidden2 = Dense(32, activation='tanh')(hidden1)
hidden3 = Dense(128, activation='tanh')(hidden2)
hidden4 = Dense(64, activation='tanh')(hidden3)


# Output layers
output1 = Dense(1, activation='tanh', name='output1')(hidden4)  # Example output, change activation and units accordingly
output2 = Dense(1, activation='tanh', name='output2')(hidden4)  # Example output, change activation and units accordingly
output3 = Dense(1, activation='tanh', name='output3')(hidden4)  # Example output, change activation and units accordingly


# Define the model
output_concat = tf.concat([output1, output2, output3], axis=1)

model = Model(inputs=inputs, outputs=output_concat)


In [251]:
from tensorflow.keras.optimizers.legacy import Adam
# Compile the model with custom loss function
#model.compile(optimizer='adam', loss={'output1': custom_loss,
#                                      'output2': custom_loss,
#                                      'output3': custom_loss,
#                                      'output4': custom_loss},
#                        metrics=['accuracy'])

#model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

class BatchNumberCallback(Callback):
    def __init__(self):
        super(BatchNumberCallback, self).__init__()
        self.batch_number = 0

    def on_batch_begin(self, batch, logs=None):
        self.batch_number = batch

batch_number_callback = BatchNumberCallback()




model.compile(optimizer='adam', loss=lambda y_true, y_pred: custom_loss(batch_number_callback, y_true, y_pred), metrics=['accuracy'])

#model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])


# Print model summary
model.summary()
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10, batch_size=32, callbacks=[batch_number_callback])

# Evaluate the model
evaluation = model.evaluate(X_test,y_test)

# Print the evaluation results
print("Evaluation results:")
for i, metric_name in enumerate(model.metrics_names):
    print(f"{metric_name}: {evaluation[i]}")

# Make predictions
predictions = model.predict(X_test)



Model: "model_79"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_80 (InputLayer)       [(None, 2)]                  0         []                            
                                                                                                  
 dense_316 (Dense)           (None, 64)                   192       ['input_80[0][0]']            
                                                                                                  
 dense_317 (Dense)           (None, 32)                   2080      ['dense_316[0][0]']           
                                                                                                  
 dense_318 (Dense)           (None, 128)                  4224      ['dense_317[0][0]']           
                                                                                           


############################## WARNING ##############################
	Predictor Version: 1.0.0
	Current Version:   1.1.0
############################## WARNING ##############################

Found 2 mismatches between original and current metadata:


[-7.90005493 0.293884277 -3.72377014 ... -2.5763855 12.8201141 -10.3039246]


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/lambda/Slice_1' defined at (most recent call last):
    File "c:\Users\mfavre4\AppData\Local\anaconda3\envs\GD_Control\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\mfavre4\AppData\Local\anaconda3\envs\GD_Control\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\mfavre4\AppData\Local\anaconda3\envs\GD_Control\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\mfavre4\AppData\Local\anaconda3\envs\GD_Control\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\mfavre4\AppData\Local\anaconda3\envs\GD_Control\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "c:\Users\mfavre4\AppData\Local\anaconda3\envs\GD_Control\lib\site-packages\tor

In [ ]:
# Plot predictions for each output
plt.figure(figsize=(12, 8))

for i, output in enumerate(['output1', 'output2', 'output3']):
    plt.subplot(3, 1, i + 1)
    plt.plot(y_test.iloc[:, i], label='True')
    plt.plot(predictions[i], label='Predicted')
    plt.title(output)
    plt.legend()

plt.tight_layout()
plt.show()